In [ ]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../')
import pandas as pd
from src.gpt_prompt import get_sentiment_general, get_sentiment_general_parallel, get_risk_score
import datetime

In [ ]:
df_news = pd.read_csv("../data/apple_2023/news_apple_2023.csv", index_col=0)
df_price = pd.read_csv("../data/apple_2023/price_apple_2023.csv", index_col=0)
df_news["date"] = pd.to_datetime(df_news['date'])
df_price["date"] = pd.to_datetime(df_price['date'])
print(df_news.head())
print(df_price.head())

In [ ]:
dfs = []
for i in range(1, 6):
    df_score = pd.read_csv(f"../data/apple_2023/news_score_apple_2023_{i}.csv", index_col=0)
    print(df_score.shape)
    df_score = df_score[["date", "score"]]
    df_score["date"] = pd.to_datetime(df_score['date'])
    dfs.append(df_score)
    
df_score = pd.concat(dfs)
df_score.reset_index(drop = True, inplace = True)
df_score.to_csv("../data/apple_2023/news_score_apple_2023_all.csv")
df_score


# df_score = pd.read_csv(f"../data/apple_2023/news_score_apple_2023_{1}.csv", index_col=0)
# print(df_score.shape)
# df_score = df_score[["date", "score"]]
# df_score["date"] = pd.to_datetime(df_score['date'])

In [ ]:
df_score = df_score.sort_values("date")
df_score.reset_index(drop = True, inplace = True)
df_score

In [ ]:
df_score["score"].value_counts()

In [ ]:
date_price_dict = df_price.set_index('date')['price'].to_dict()
# date_price_dict

In [ ]:
def find_whole_minute_time_after_n_minutes(time, n):
    new_time = time + pd.Timedelta(minutes=n)
    if new_time.second > 0:
        new_time += pd.Timedelta(minutes=1)
        new_time = new_time.replace(second=0, microsecond=0)
    return new_time

In [ ]:
df_news.shape

In [ ]:

columns = [f"price_in_{i}" for i in range(61)]
deltas = [i for i in range(61)]
pd_price = np.zeros((df_news.shape[0], 61))

for i in range(df_news.shape[0]):
    time = df_news["date"][i]
    for j in range(61):
        delta = deltas[j]
        new_time = find_whole_minute_time_after_n_minutes(time, delta)
        if new_time in date_price_dict:
            pd_price[i, j] = date_price_dict[new_time]

In [ ]:
date_price_dict

In [ ]:
pd_price = pd.DataFrame(pd_price, columns=columns)
result_df = pd.concat([df_news, pd_price], axis=1)

In [ ]:
result_df

In [ ]:
df_score

In [ ]:
result_df.replace(0, pd.NA, inplace=True)
result_df = result_df.merge(df_score, on='date', how='left')
result_df["score"] = result_df["score"].astype(float)
result_df = result_df.dropna()



return_cols = [f"return_in_{i}" for i in range(1,61)]
price_cols = [f"price_in_{i}" for i in range(61)]

for i in range(60):
    result_df[return_cols[i]] = (result_df[price_cols[i]] - result_df["price_in_0"]) / result_df["price_in_0"]
    result_df[return_cols[i]] = result_df[return_cols[i]].astype(float)
for col in price_cols:
    result_df.drop(col, axis=1, inplace=True)
result_df = result_df.dropna()
result_df

In [ ]:
result_df = result_df.dropna()

In [ ]:
result_df.describe(include='all')

In [ ]:
result_df[result_df['score'] == i][f"return_in_{j}"].mean()

In [ ]:
# import seaborn as sns
# for return_col in ['return_in_5', 'return_in_10', 'return_in_20', 'return_in_60']:
#     sns.scatterplot(x='score', y=return_col, data=result_df, alpha=0.1)
#     plt.title(f'Score vs {return_col}')
#     plt.show()

In [ ]:
import matplotlib.pyplot as plt
for i in [-1, 0, 1]:
    arr = []
    for j in range(1, 61):
        avg_return = result_df[result_df['score'] == i][f"return_in_{j}"].mean()
        arr.append(avg_return)
    plt.plot(list(range(1,61)), arr, label=f'Label {i}')

plt.legend()
plt.show()
    

In [ ]:

for j in [-1, 0, 1]:
    print(f"When score is {j}, the mean return is {result_df[result_df['score'] == j]['return_in_60'].mean()*10000}")
    